In [2]:
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format

df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\stock.xlsx") 
df_stock.drop([0])
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory.xlsx")
df_CS = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\Credit_S.xlsx")


In [3]:
#column filter
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)

ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
final_prepayment = pd.merge(ffca_qv, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

final_prepayment.columns

Index(['Purchase Assignment ID', 'Commodity', 'Inventory ID',
       'Counterparty of Purchase', 'Purchase Settled Amount',
       'Authorised Paid Quantity', 'Financed Quantity', 'Current Value',
       'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount',
       'Assignment ID', 'Group Company', 'Trade ID', 'Quota ID',
       'Traffic Operator', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT',
       'WET WEIGHT', 'EXPECTED RECEIPT DATE', 'Inventory Id',
       'Origin Country'],
      dtype='object')

In [4]:
final_prepayment.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)',  'Authorized Paid Quantity', 'Financed Quantity','Inventory Current Value(USD)',\
          'Purchase First Paid Date', 'Purchase Invoice ID', 'Purchase Invoice Amount(USD)','Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'INVENTORY ID', \
                      'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight', 'Purchase Expected Receipt Date', 'Inventory Id', 'Purchase Material Origin Country']

In [11]:
#增加列
final_prepayment['Purchase Covered Amount(USD)'] = '0'
final_prepayment['Purchase Dry Weight UOM'] = 'MT'
final_prepayment['Purchase Wet Weight UOM'] = 'MT'
final_prepayment['Receipt In Progress'] = 'N'
final_prepayment['Delivery In Progress'] = 'N'
final_prepayment['Authorized Paid Quantity'] = pd.to_numeric(final_prepayment['Authorized Paid Quantity'], errors='coerce')
final_prepayment['Financed Quantity'] = pd.to_numeric(final_prepayment['Financed Quantity'], errors='coerce')

# final_prepayment[['Authorized Paid Quantity', 'Financed Quantity']] = final_prepayment[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
final_prepayment.loc[:,"Financing Ratio"] = final_prepayment["Authorized Paid Quantity"]/final_prepayment["Financed Quantity"]

final_prepayment.loc[:, 'Purchase Paid Amount(USD)'] = final_prepayment['Purchase Invoice Amount(USD)']

In [12]:
#复制列
final_prepayment.loc[:, 'Eligible Dry Weight'] = final_prepayment['Purchase Dry Weight']
final_prepayment.loc[:, 'Eligible Wet Weight'] = final_prepayment['Purchase Wet Weight']

In [13]:
#删除列
final_prepayment.drop(columns=[ 'Assignment ID', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Inventory Id'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty',  \
        'Purchase Commodity', \
             'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Material Origin Country', \
                'Purchase Settlement Valuation(USD)', 'Purchase First Paid Date', 'Purchase Expected Receipt Date', 'Purchase Invoice ID', \
                    'Purchase Invoice Amount(USD)', 'Purchase Paid Amount(USD)', 'Authorized Paid Quantity', 'Financed Quantity', 'Financing Ratio', \
                        'Purchase Covered Amount(USD)', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

In [14]:
final_prepayment = final_prepayment[order]
final_prepayment.head()

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Purchase Dry Weight,Purchase Dry Weight UOM,...,Purchase Invoice Amount(USD),Purchase Paid Amount(USD),Authorized Paid Quantity,Financed Quantity,Financing Ratio,Purchase Covered Amount(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight
0,PTE,651719.00,651719.28,651719.28.7,879568,Ajit Braganza,Anvil Mining Congo SARL,Copper Concentrates,590.89,MT,...,225223.92,225223.92,136,136.10,1.00,0,N,N,590.89,671.17
1,PTE,651719.00,651719.28,651719.28.7,879568,Ajit Braganza,Anvil Mining Congo SARL,Copper Concentrates,590.89,MT,...,225223.92,225223.92,136,136.10,1.00,0,N,N,590.89,671.17
2,PTE,651719.00,651719.29,651719.29.3,879568,Ajit Braganza,Anvil Mining Congo SARL,Copper Concentrates,590.89,MT,...,831575.22,831575.22,450,450.20,1.00,0,N,N,590.89,671.17
3,PTE,651719.00,651719.29,651719.29.3,879568,Ajit Braganza,Anvil Mining Congo SARL,Copper Concentrates,590.89,MT,...,831575.22,831575.22,450,450.20,1.00,0,N,N,590.89,671.17
4,PTE,661875.00,661875.15,661875.15.72,877159,Keagan Pearson,Khoemacau Copper Mining Pty Ltd,Copper Concentrates,29.07,MT,...,10.48,10.48,0,0.00,0.00,0,N,N,29.07,32.10


In [18]:
input = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\input.xlsx")
result2 = pd.merge(final_prepayment, input, left_on=["Purchase Assignment Reference"], right_on=["assignment"], how='right')
result2 = result2.drop_duplicates(['Purchase Assignment Reference'])
result2

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Purchase Dry Weight,Purchase Dry Weight UOM,...,Purchase Paid Amount(USD),Authorized Paid Quantity,Financed Quantity,Financing Ratio,Purchase Covered Amount(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,assignment
0,PTE,647549.00,647549.90,647549.9.12,847590,Micaela DeSouza,EMED Mining Public Ltd,Copper Concentrates,9610.09,MT,...,4336911.37,2787,2787.18,1.00,0,N,N,9610.09,10677.91,647549.9.12
2,PTE,647549.00,647549.90,647549.9.6,834791,Nayelli Rosales,EMED Mining Public Ltd,Copper Concentrates,2419.96,MT,...,1887182.36,1210,1209.98,1.00,0,N,N,2419.96,2695.14,647549.9.6
4,PTE,647556.00,647556.24,647556.24.3,805457,Alkistis Korali,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,2514.19,MT,...,4067195.42,2510,2509.73,1.00,0,N,N,2514.19,2742.95,647556.24.3


In [20]:
import numpy as np
test_p = result2.copy(deep=True)

test_p['Purchase Contractual Counterparty'] = np.nan
test_p['Purchase Settlement Valuation(USD)'] = np.nan
test_p['Authorized Paid Quantity'] = np.nan
test_p['Financed Quantity'] = np.nan
test_p['Financing Ratio'] = np.nan
test_p['Purchase Covered Amount(USD)'] = np.nan
test_p['Purchase Uncovered Amount(USD)'] = np.nan
test_p['Receipt In Progress'] = np.nan
test_p['Delivery In Progress'] = np.nan
test_p['Eligible Dry Weight'] = np.nan
test_p['Eligible Wet Weight'] = np.nan


test_q = result2.copy(deep=True)
test_q['Purchase Invoice ID'] = np.nan


test_f = pd.concat([test_p,test_q])
test_f = test_f.sort_values(by=['Purchase Assignment Reference'], ascending=False)
test_f

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Purchase Dry Weight,Purchase Dry Weight UOM,...,Authorized Paid Quantity,Financed Quantity,Financing Ratio,Purchase Covered Amount(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,assignment,Purchase Uncovered Amount(USD)
4,PTE,647556.00,647556.24,647556.24.3,805457,Alkistis Korali,NaN,Copper Concentrates,2514.19,MT,...,nan,nan,nan,NaN,NaN,NaN,nan,nan,647556.24.3,nan
4,PTE,647556.00,647556.24,647556.24.3,805457,Alkistis Korali,Zangezur Copper-Molybdenum Combine CJSC,Copper Concentrates,2514.19,MT,...,2510.00,2509.73,1.00,0,N,N,2514.19,2742.95,647556.24.3,nan
2,PTE,647549.00,647549.90,647549.9.6,834791,Nayelli Rosales,NaN,Copper Concentrates,2419.96,MT,...,nan,nan,nan,NaN,NaN,NaN,nan,nan,647549.9.6,nan
2,PTE,647549.00,647549.90,647549.9.6,834791,Nayelli Rosales,EMED Mining Public Ltd,Copper Concentrates,2419.96,MT,...,1210.00,1209.98,1.00,0,N,N,2419.96,2695.14,647549.9.6,nan
0,PTE,647549.00,647549.90,647549.9.12,847590,Micaela DeSouza,NaN,Copper Concentrates,9610.09,MT,...,nan,nan,nan,NaN,NaN,NaN,nan,nan,647549.9.12,nan
0,PTE,647549.00,647549.90,647549.9.12,847590,Micaela DeSouza,EMED Mining Public Ltd,Copper Concentrates,9610.09,MT,...,2787.00,2787.18,1.00,0,N,N,9610.09,10677.91,647549.9.12,nan
